# **Chapter 20: Data Splitting Strategies**

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Understand why standard random splitting fails for time‑series data
- Recognize the problems caused by data leakage in temporal settings
- Implement proper train‑validation‑test splits using time‑based cutoffs
- Apply walk‑forward, rolling window, and expanding window validation techniques
- Use purging and embargoing to prevent leakage in cross‑validation
- Execute time‑series cross‑validation with `TimeSeriesSplit` and its variants
- Handle multiple assets (symbols) correctly when splitting
- Choose the right splitting strategy for your forecasting problem
- Avoid common pitfalls that lead to overoptimistic performance estimates

---

## **20.1 Why Time‑Series Splitting is Different**

In standard machine learning, data points are assumed to be independent and identically distributed (i.i.d.). Therefore, we can randomly shuffle the data and split it into training, validation, and test sets without worrying about temporal order. However, time‑series data violates the i.i.d. assumption because observations are ordered in time and exhibit autocorrelation, trends, and seasonality.

Using a random split on time‑series data would allow the model to be trained on future data and tested on past data, which is impossible in a real‑world deployment. This leads to **data leakage**: the model sees information from the future during training, artificially inflating performance metrics. When deployed, the model fails because it cannot access future data.

For the NEPSE prediction system, a random split might place January 2024 data in the training set and December 2023 data in the test set. The model would then "learn" patterns that rely on future events, like a market reaction to a budget announcement that happened in June, and then "predict" prices in December as if it knew the budget outcome. This is nonsense and would never work in live trading.

Therefore, we must always respect the temporal order: **training data must come before validation data, which must come before test data**.

### **20.1.1 The Arrow of Time**

The fundamental principle of time‑series splitting is that we can only use past information to predict the future. When we split, we choose a cutoff date. All data before that date is used for training; all data after is used for validation or testing. This mimics the real scenario where we train a model on historical data and then use it to forecast future, unseen periods.

```python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load NEPSE data
df = pd.read_csv('nepse_data.csv')
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(['Symbol', 'Date']).reset_index(drop=True)

# For simplicity, we'll work with one symbol (e.g., 'NEPSE' or a specific stock)
symbol = df['Symbol'].unique()[0]  # pick first symbol
df_one = df[df['Symbol'] == symbol].copy()

# Plot the closing price to see the time order
plt.figure(figsize=(12, 4))
plt.plot(df_one['Date'], df_one['Close'])
plt.axvline(x=pd.to_datetime('2023-06-01'), color='r', linestyle='--', label='Potential split date')
plt.xlabel('Date')
plt.ylabel('Close Price (NPR)')
plt.title(f'{symbol} - Closing Price Over Time')
plt.legend()
plt.show()
```

**Explanation:**

- This plot shows the natural progression of time. Any split must preserve this order. The red dashed line indicates a possible cutoff: data to the left (past) is training, to the right (future) is testing.
- In practice, we would choose a cutoff that leaves enough test data for reliable evaluation (e.g., the last 20% of the period).

---

## **20.2 Random Split Problems**

Let's demonstrate why random splitting is disastrous for time‑series. We'll create a simple feature (lagged return) and target (next day's return), then perform a random split and evaluate a model. The performance will look artificially good because the model uses future information.

```python
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Create a simple feature: yesterday's return
df_one['Return'] = df_one['Close'].pct_change()
df_one['Feature'] = df_one['Return'].shift(1)  # yesterday's return
df_one['Target'] = df_one['Return'].shift(-1)  # tomorrow's return

# Drop NaN rows
df_ml = df_one[['Feature', 'Target']].dropna()

# Random split (WRONG for time series)
X = df_ml[['Feature']]
y = df_ml['Target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a model
model = RandomForestRegressor(n_estimators=50, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Random split RMSE: {rmse:.6f}")

# Check temporal order in test set
test_indices = X_test.index
print(f"Test set indices (should be random): {sorted(test_indices)[:10]}")
```

**Explanation:**

- The random split shuffles the data, so the training set contains both early and late observations, and the test set also contains a mix. This means the model can learn patterns that depend on the future relative to some training points. For example, if a similar pattern occurred in both 2022 and 2023, the model might see the 2023 pattern during training and then be tested on the 2022 pattern – but in reality, we can't know 2023 when predicting 2022.
- The RMSE might look reasonable, but it's an illusion. When we deploy such a model, it will fail because future data is not available at prediction time.
- The printed indices confirm that the test set includes a mix of early and late dates – exactly the problem.

---

## **20.3 Train‑Validation‑Test Split**

The standard approach in machine learning is to have three sets: training (for model fitting), validation (for hyperparameter tuning and model selection), and test (for final unbiased evaluation). For time‑series, we create these by choosing two cutoff dates.

```python
# Define cutoff dates
train_end = '2022-12-31'
val_end = '2023-06-30'

# Split
train = df_one[df_one['Date'] < train_end]
val = df_one[(df_one['Date'] >= train_end) & (df_one['Date'] < val_end)]
test = df_one[df_one['Date'] >= val_end]

print(f"Train: {train['Date'].min()} to {train['Date'].max()} ({len(train)} rows)")
print(f"Validation: {val['Date'].min()} to {val['Date'].max()} ({len(val)} rows)")
print(f"Test: {test['Date'].min()} to {test['Date'].max()} ({len(test)} rows)")

# Now create features and targets for each set
def prepare_data(df, feature_cols, target_col):
    df = df.copy()
    # Create features and target (assuming they are already computed)
    X = df[feature_cols]
    y = df[target_col]
    return X, y

# Example: use lagged return as feature, next day's return as target
feature_cols = ['Return_Lag1']  # assume we created this column
target_col = 'Target_Return_t+1'

# Ensure columns exist (we need to compute them for each split separately to avoid leakage)
# For demonstration, we compute them globally first, but in practice compute within each split.
# Here we assume they are already in df_one.
X_train, y_train = prepare_data(train, feature_cols, target_col)
X_val, y_val = prepare_data(val, feature_cols, target_col)
X_test, y_test = prepare_data(test, feature_cols, target_col)
```

**Explanation:**

- We choose two dates: one to mark the end of training, another to mark the end of validation. Everything after the second date is test.
- The training set is the earliest period. The validation set comes next, and the test set is the most recent.
- This respects the temporal order: the model is trained on past data, tuned on a later period (but still before test), and finally evaluated on the most recent, unseen data.
- When computing features that require lookback (e.g., rolling means), we must ensure that for the validation and test sets, we only use information available up to that point. In practice, this means we should not compute features globally; instead, we should compute them within each split using only the data available at that time. For example, for a validation point, the rolling mean should be computed using only data from before that point (including training data, but not future validation data). This is a subtle but important point – we'll address it later.

---

## **20.4 Time‑Based Splitting**

Time‑based splitting is the simplest and most common method for time‑series. You choose a single cutoff date and split into training and test (or training, validation, and test as above). The key is to ensure the test set is chronologically after the training set.

### **20.4.1 Simple Single Cutoff**

```python
cutoff_date = '2023-01-01'
train = df_one[df_one['Date'] < cutoff_date]
test = df_one[df_one['Date'] >= cutoff_date]

print(f"Train size: {len(train)}, Test size: {len(test)}")
```

### **20.4.2 Multiple Cutoffs for Train/Val/Test**

As shown in 20.3, we can use two cutoffs to create three sets. This is typical when you need to tune hyperparameters.

### **20.4.3 Importance of Gap**

Sometimes, to avoid any potential leakage from very recent history, we introduce a gap between training and validation/test. For example, if we use features that require a lookback window (e.g., 20-day moving average), we might want to ensure that the validation set does not include the first few points that depend on training data that is too close to the cutoff. But careful: if we skip data, we lose information. Usually, it's fine to have contiguous splits as long as we compute features correctly.

```python
# Introduce a 1-month gap between train and validation
gap_start = '2023-01-01'
gap_end = '2023-02-01'

train = df_one[df_one['Date'] < gap_start]
gap = df_one[(df_one['Date'] >= gap_start) & (df_one['Date'] < gap_end)]  # discard
val = df_one[df_one['Date'] >= gap_end]
```

**Explanation:**

- The gap period is not used for either training or validation. This can be useful if there is a known structural break (e.g., a regulatory change) or to ensure that validation truly starts after all training‑derived features are stable. However, it reduces the amount of data available.

---

## **20.5 Walk‑Forward Validation**

Walk‑forward validation (also known as rolling window validation) simulates how a model would be used in practice: we train on an initial window, then predict the next period, then expand the window (or move it) and repeat. This gives multiple performance estimates and helps assess model stability over time.

### **20.5.1 Expanding Window Walk‑Forward**

In expanding window, we start with an initial training set, then add more data as we move forward.

```python
# Define initial training size and test size (in days)
initial_train_days = 500
test_days = 50
step = 50  # move forward by 50 days each time

dates = df_one['Date'].values
total_days = len(dates)

scores = []
start = 0

while start + initial_train_days + test_days <= total_days:
    train_end = start + initial_train_days
    test_end = train_end + test_days
    
    train_indices = range(start, train_end)
    test_indices = range(train_end, test_end)
    
    X_train = df_one.iloc[train_indices][['Feature']]  # feature
    y_train = df_one.iloc[train_indices]['Target']
    X_test = df_one.iloc[test_indices][['Feature']]
    y_test = df_one.iloc[test_indices]['Target']
    
    model = RandomForestRegressor(n_estimators=50, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    scores.append(rmse)
    
    print(f"Window {start}: train {dates[start]} to {dates[train_end-1]}, "
          f"test {dates[train_end]} to {dates[test_end-1]}, RMSE: {rmse:.6f}")
    
    start += step

print(f"Average RMSE across windows: {np.mean(scores):.6f}")
```

**Explanation:**

- We slide a window of fixed test size across the data. The training set expands each time because we keep all previous data (`start` increases, but we always train from the beginning up to `train_end`).
- This mimics a realistic retraining scenario: as new data arrives, we retrain on all available history and forecast the next period.
- The average RMSE gives a robust estimate of model performance under different market conditions. If scores vary widely, the model is unstable.

### **20.5.2 Rolling Window Walk‑Forward**

In rolling window, the training set size remains fixed; we drop the oldest data as we add new data.

```python
window_size = 500
test_days = 50
step = 50

scores_rolling = []

for start in range(0, total_days - window_size - test_days, step):
    train_start = start
    train_end = start + window_size
    test_end = train_end + test_days
    
    train_indices = range(train_start, train_end)
    test_indices = range(train_end, test_end)
    
    # Same fitting as above...
    X_train = df_one.iloc[train_indices][['Feature']]
    y_train = df_one.iloc[train_indices]['Target']
    X_test = df_one.iloc[test_indices][['Feature']]
    y_test = df_one.iloc[test_indices]['Target']
    
    model = RandomForestRegressor(n_estimators=50, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    scores_rolling.append(rmse)

print(f"Rolling window average RMSE: {np.mean(scores_rolling):.6f}")
```

**Explanation:**

- Rolling window is useful when the relationship between features and target may change over time (concept drift). By using only recent data, the model can adapt to new regimes.
- However, it discards older data, which might contain valuable long‑term patterns. For the NEPSE market, where structural changes (e.g., new regulations) occur, rolling window might be more appropriate.
- The choice between expanding and rolling depends on the stationarity of the data.

---

## **20.6 Rolling Window Validation**

Rolling window validation is essentially the same as walk‑forward with a fixed training window. The term is often used interchangeably. In the context of hyperparameter tuning, we might use a rolling window approach to evaluate different parameter sets.

### **20.6.1 Example with Hyperparameter Tuning**

```python
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

# We'll define a custom time-series splitter for rolling windows
class RollingWindowSplit:
    def __init__(self, window_size, test_size, step=1):
        self.window_size = window_size
        self.test_size = test_size
        self.step = step
    
    def split(self, X, y=None, groups=None):
        n_samples = len(X)
        for start in range(0, n_samples - self.window_size - self.test_size, self.step):
            train_end = start + self.window_size
            test_end = train_end + self.test_size
            train_indices = list(range(start, train_end))
            test_indices = list(range(train_end, test_end))
            yield train_indices, test_indices

# Assume X and y are the full dataset (with features and target)
window_split = RollingWindowSplit(window_size=500, test_size=50, step=50)

# Use this split in GridSearchCV
param_grid = {'n_estimators': [50, 100], 'max_depth': [5, 10]}
model = RandomForestRegressor(random_state=42)

# Note: GridSearchCV expects a cross-validation generator; we can use our custom splitter
# But we need to ensure it yields indices correctly. We'll use a simple loop instead for clarity.
best_score = -np.inf
best_params = None

for params in [{'n_estimators': 50, 'max_depth': 5}, {'n_estimators': 100, 'max_depth': 10}]:
    scores = []
    for train_idx, test_idx in window_split.split(X):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        model.set_params(**params)
        model.fit(X_train, y_train)
        scores.append(model.score(X_test, y_test))  # R²
    avg_score = np.mean(scores)
    if avg_score > best_score:
        best_score = avg_score
        best_params = params

print(f"Best params: {best_params}, average validation R²: {best_score:.4f}")
```

**Explanation:**

- We define a custom splitter that yields training and test indices for each rolling window.
- We then loop over hyperparameter combinations, compute the average performance across windows, and select the best.
- This gives a robust estimate of how the model would perform over time, reducing the risk of overfitting to a specific validation period.

---

## **20.7 Expanding Window Validation**

Expanding window validation is similar, but the training set grows over time.

```python
class ExpandingWindowSplit:
    def __init__(self, min_train_size, test_size, step=1):
        self.min_train_size = min_train_size
        self.test_size = test_size
        self.step = step
    
    def split(self, X, y=None, groups=None):
        n_samples = len(X)
        for train_end in range(self.min_train_size, n_samples - self.test_size, self.step):
            train_indices = list(range(train_end))
            test_indices = list(range(train_end, train_end + self.test_size))
            yield train_indices, test_indices

# Use similar to above
```

**Explanation:**

- Expanding window is appropriate when you believe that more data always helps (i.e., the process is stationary) and you want to use all available history.
- In financial markets, both expanding and rolling are used; expanding is more common for long‑term trend models, rolling for short‑term adaptive models.

---

## **20.8 Purging and Embargoing**

When using cross‑validation that involves overlapping windows (like rolling windows), there is a risk of leakage between training and test sets if the test set contains data that is temporally close to the training set. For example, if you have a 20‑day moving average feature, the first few days of the test set might depend on data from the end of the training set. This is a form of leakage because the feature for the first test day uses training data, which is fine, but if the same data point appears in both training and test sets? Actually, no – they are separate indices. The issue is more subtle: when you have autocorrelated errors, the test error may be underestimated if the test set immediately follows the training set because the market conditions are similar.

To mitigate this, we introduce **purging** and **embargoing**.

- **Purging** means removing from the training set any data that overlaps in time with the test set (e.g., if you use lagged features, the first few test observations might have features computed from data that is also in the training set? Actually no, because features are computed only from past data. The real issue is that if the test set includes the day immediately after the training set, the model might be evaluated on a day that is very similar to the last training day, giving an optimistic estimate. This is not leakage per se, but it can lead to overfitting to the most recent regime.
- **Embargoing** means excluding from the training set a period immediately before the test set (e.g., a few days or weeks) to ensure that the test set is truly out‑of‑sample and not too similar to the training data.

In practice, purging is often automatic if we compute features correctly. Embargoing is a common practice in financial cross‑validation.

```python
# Example of embargo: exclude a gap before each test set
embargo_days = 5

class RollingWindowWithEmbargo:
    def __init__(self, window_size, test_size, embargo, step=1):
        self.window_size = window_size
        self.test_size = test_size
        self.embargo = embargo
        self.step = step
    
    def split(self, X, y=None, groups=None):
        n_samples = len(X)
        for start in range(0, n_samples - self.window_size - self.test_size, self.step):
            train_end = start + self.window_size
            # Apply embargo: remove last 'embargo' days from training
            train_indices = list(range(start, train_end - self.embargo))
            test_indices = list(range(train_end, train_end + self.test_size))
            if len(train_indices) > 0:
                yield train_indices, test_indices
```

**Explanation:**

- By removing the last `embargo` days from the training set before the test set, we ensure that the test set is not immediately adjacent to the training data. This reduces the chance of overfitting to short‑term autocorrelation.
- The embargo period is discarded from training (not used in either set). This is a conservative approach that simulates a real‑world gap (e.g., we train on data up to a point, then wait a few days before making predictions).

---

## **20.9 Cross‑Validation for Time‑Series**

Cross‑validation (CV) is a resampling technique to estimate model performance. For time‑series, we cannot use standard k‑fold CV because it shuffles the data. Instead, we use specialized methods.

### **20.9.1 TimeSeriesSplit (Scikit‑Learn)**

Scikit‑learn provides `TimeSeriesSplit`, which is an expanding window cross‑validator.

```python
from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(n_splits=5)

for fold, (train_idx, test_idx) in enumerate(tscv.split(X)):
    print(f"Fold {fold+1}:")
    print(f"  Train indices: {train_idx[0]}:{train_idx[-1]} (size {len(train_idx)})")
    print(f"  Test indices: {test_idx[0]}:{test_idx[-1]} (size {len(test_idx)})")
    # Train and evaluate model here
```

**Explanation:**

- `TimeSeriesSplit` creates successive training sets that are supersets of previous ones (expanding window). The test sets are non‑overlapping and sequential.
- This is a good default for time‑series CV, but it does not include an embargo. For many applications, it's sufficient.

### **20.9.2 Blocked Cross‑Validation**

Blocked CV divides the time series into contiguous blocks and uses some blocks for training, others for testing, in a way that respects order. For example, we might use the first 3 blocks for training, the next block for testing, then move forward.

```python
from sklearn.model_selection import PredefinedSplit

# Example: create 5 blocks of equal size
n_blocks = 5
block_size = len(X) // n_blocks

test_fold = np.full(len(X), -1)  # -1 indicates training

for i in range(1, n_blocks):
    test_start = i * block_size
    test_end = (i+1) * block_size if i < n_blocks-1 else len(X)
    test_fold[test_start:test_end] = i  # fold number for test

# Now PredefinedSplit will use these assignments
ps = PredefinedSplit(test_fold)

for train_idx, test_idx in ps.split():
    print(f"Train indices: {train_idx[:5]}...{train_idx[-5:]}")
    print(f"Test indices: {test_idx[:5]}...{test_idx[-5:]}")
```

**Explanation:**

- This approach creates non‑overlapping test sets. It can be useful when you want to evaluate on distinct time periods.
- However, it may not be ideal if the time series is short because the test sets may be small.

### **20.9.3 Modified Cross‑Validation (Purged, Embargoed)**

The financial literature (e.g., Marcos López de Prado's "Advances in Financial Machine Learning") introduces purged and embargoed cross‑validation. The idea is to remove from the training set any data that overlaps in time with the test set due to feature lags (purging), and also to exclude a gap (embargo) to prevent leakage from very recent observations.

Implementing full purged CV is complex and beyond the scope of this chapter, but we can approximate it with the embargo idea shown earlier.

---

## **20.10 Multiple Asset Splitting**

When we have multiple symbols (stocks) in the dataset, we must decide how to split. The options are:

1. **Split by time globally:** Choose a cutoff date and use that for all symbols. This ensures that the model is tested on the same time period for all stocks. This is the most common approach.
2. **Split by symbol:** Train on some symbols, test on others. This tests cross‑sectional generalization (whether patterns learned on one stock apply to another). This is less common for time‑series forecasting but can be useful if you have many stocks.
3. **Split by both time and symbol:** For example, train on symbols A, B, C for years 2010‑2019, test on symbol D for 2020. This is a more rigorous test.

For the NEPSE system, we likely want to predict each symbol individually, so a time‑based split applied per symbol is appropriate. However, we must ensure that for each symbol, the split dates are aligned (e.g., use the same calendar cutoff).

```python
# Define a global cutoff date
cutoff_date = '2023-01-01'

# Create train/test for each symbol
train_list = []
test_list = []

for symbol, group in df.groupby('Symbol'):
    train = group[group['Date'] < cutoff_date].copy()
    test = group[group['Date'] >= cutoff_date].copy()
    train_list.append(train)
    test_list.append(test)

train_all = pd.concat(train_list)
test_all = pd.concat(test_list)

print(f"Total train: {len(train_all)}, Total test: {len(test_all)}")
```

**Explanation:**

- This preserves the temporal order per symbol and uses the same cutoff for all. The model will be trained on a mix of symbols from earlier periods and tested on later periods.
- When computing features that require lookback (e.g., rolling means), we must compute them within each symbol group separately, as we did in Chapter 19.

---

## **20.11 Splitting Best Practices**

To wrap up, here are the key best practices for splitting time‑series data, especially for the NEPSE prediction system:

### **20.11.1 Always Respect Temporal Order**

Never shuffle your data before splitting. Use date‑based cutoffs.

### **20.11.2 Use Expanding or Rolling Windows for Validation**

For hyperparameter tuning, use walk‑forward validation (expanding or rolling) rather than a single validation set. This gives a more robust estimate.

### **20.11.3 Purge and Embargo When Necessary**

If your features have long lookback windows, consider adding an embargo period to avoid evaluating on data too similar to the training set.

### **20.11.4 Maintain Consistent Feature Computation**

Compute features within each training fold using only the data available up to that point. Do not use global statistics. This is especially important for rolling means, normalization, and any transformation that depends on the data distribution.

```python
# Example: correct way to compute rolling mean within a training fold
def compute_features_for_fold(train_idx, test_idx, df, window=20):
    # Combine train and test in order, but we will only use train data for the rolling calc
    combined = df.iloc[np.r_[train_idx, test_idx]].copy()
    
    # Compute rolling mean using expanding window that only sees past data
    # For training indices, we need rolling mean that includes only previous training points
    # This is tricky; simpler: compute on the full combined but ensure that for test,
    # the rolling mean uses data only up to that point (including train). This is fine.
    combined['rolling_mean'] = combined['Close'].rolling(window, min_periods=1).mean()
    
    # Now split back
    train_features = combined.iloc[:len(train_idx)][['rolling_mean']]
    test_features = combined.iloc[len(train_idx):][['rolling_mean']]
    return train_features, test_features
```

### **20.11.5 Document Your Splitting Strategy**

Always record the split dates and method used. This ensures reproducibility and helps diagnose why a model might fail in production.

### **20.11.6 Test on the Most Recent Data**

Your final evaluation should always be on the most recent data available, as this best represents the conditions the model will face in deployment.

---

## **Chapter Summary**

In this chapter, we explored the critical topic of data splitting for time‑series prediction systems. Using the NEPSE dataset, we demonstrated:

- **Why random splitting fails** – it leads to data leakage and overoptimistic performance.
- **Time‑based splitting** – using cutoff dates to create train, validation, and test sets that respect the arrow of time.
- **Walk‑forward validation** – expanding and rolling windows that simulate realistic retraining scenarios.
- **Purging and embargoing** – techniques to further reduce leakage from temporal proximity.
- **Time‑series cross‑validation** – using `TimeSeriesSplit` and custom splitters.
- **Multiple asset splitting** – handling multiple symbols with consistent time cutoffs.
- **Best practices** – a checklist for robust evaluation.

### **Practical Takeaways for the NEPSE System:**

- Always split by time, never randomly.
- Use walk‑forward validation to tune models and estimate performance over different market regimes.
- For final evaluation, use a hold‑out set from the most recent period.
- When computing features, ensure they are calculated using only data available at the time (no lookahead).
- If you have many symbols, apply the same temporal split to all, but compute features per symbol.

With a solid splitting strategy, we can now confidently evaluate models. In the next chapter, **Chapter 21: Traditional Statistical Models**, we will apply these splitting techniques to evaluate classical time‑series models like ARIMA, Exponential Smoothing, and others on the NEPSE data.

---

**End of Chapter 20**

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='19. defining_prediction_targets.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='21. traditional_statistical_models.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
